## Initialize global variable

In [1]:
# hosts=tf gather_facts=no
- set_fact: 
    pet_resource_path="/res/material/robots.ox.ac.uk/pets"
    models_resource_path="/res/material/tensorflow/models/object_detection"
    od_pet_project_path="$HOME/tftest/od_pet"
    od_project_prefix="$HOME/tensorflow/models"

Set hosts to: tf	Set gather_facts to: no	
Use hosts: tf	Use gather_facts: no	
TASK [192.168.10.238: TASK: set_fact] 


ok

## Getting the Oxford-IIIT Pets Dataset

### Download images and annotations file from remote and tar files

In [25]:
- get_url:
    url: http://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
    dest: '{{ pet_resource_path }}/images.tar.gz'

- get_url:
    url: http://www.robots.ox.ac.uk/~vgg/data/pets/data/annotations.tar.gz
    dest: '{{ pet_resource_path }}/annotations.tar.gz'

- stat: path='{{ od_pet_project_path }}/images'
  register: pet_images_path_stat

- shell: |
    . ~/.zshrc
    cd {{od_pet_project_path}}
    cp {{ pet_resource_path }}/images.tar.gz .
    cp {{ pet_resource_path }}/annotations.tar.gz .
    tar -xf images.tar.gz
    tar -xf annotations.tar.gz
    rm images.tar.gz annotations.tar.gz
  args:
    executable: /bin/zsh
  when: pet_images_path_stat.stat.exists == False


Use hosts: tf	Use gather_facts: no	
TASK [192.168.10.238: TASK: get_url] 
file already exists

TASK [192.168.10.238: TASK: get_url] 
file already exists

TASK [192.168.10.238: TASK: stat] 
changed: False

ok

### Run 'create_pet_tf_record' to convert from the raw Oxford-IIIT Pet dataset into TFRecords.

In [26]:
- shell: |
    . ~/.zshrc
    python object_detection/create_pet_tf_record.py \
        --label_map_path=object_detection/data/pet_label_map.pbtxt \
        --data_dir={{ od_pet_project_path }} \
        --output_dir={{ od_pet_project_path }}
  args:
    chdir: '{{ od_project_prefix }}'
    executable: /bin/zsh

Use hosts: tf	Use gather_facts: no	
TASK [192.168.10.238: TASK: command] 
stdout:

stderr:
/home/mind/tensorflow/models/object_detection/utils/dataset_util.py:75: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not xml:

ok

### move pet_train.record and pet_val.record to data

In [35]:
- shell: |
    . ~/.zshrc
    cd {{ od_pet_project_path }}
    mkdir data
    mv *.record data
    cp {{od_project_prefix}}/object_detection/data/pet_label_map.pbtxt data/pet_label_map.pbtxt
  args:
    executable: /bin/zsh

Use hosts: tf	Use gather_facts: no	
TASK [192.168.10.238: TASK: command] 
stdout:

stderr:
mkdir: cannot create directory ‘data’: File exists
zsh:4: no matches found: *.record

ok

## Downloading a COCO-pretrainsed Model for Transfer Learning

In [33]:
- get_url:
    url: http://storage.googleapis.com/download.tensorflow.org/models/object_detection/faster_rcnn_resnet101_coco_11_06_2017.tar.gz
    dest: '{{ models_resource_path }}/faster_rcnn_resnet101_coco_11_06_2017.tar.gz'

- shell: 'ls {{od_pet_project_path}}/data/model.ckpt.*'
  register: pet_model_file_stat
  ignore_errors: true

- shell: |
    . ~/.zshrc
    cd {{od_pet_project_path}}
    cp {{ models_resource_path }}/faster_rcnn_resnet101_coco_11_06_2017.tar.gz .
    tar -xf faster_rcnn_resnet101_coco_11_06_2017.tar.gz
    cp faster_rcnn_resnet101_coco_11_06_2017/model.ckpt.* data
    rm -rf faster_rcnn_resnet101_coco_11_06_2017
    rm faster_rcnn_resnet101_coco_11_06_2017.tar.gz
  args:
    executable: /bin/zsh
  when: '{{ pet_model_file_stat|failed }}'

Use hosts: tf	Use gather_facts: no	
TASK [192.168.10.238: TASK: get_url] 
file already exists

TASK [192.168.10.238: TASK: command] 
stdout:
/home/mind/tftest/od_pet/data/model.ckpt.data-00000-of-00001
/home/mind/tftest/od_pet/data/model.ckpt.index
/home/mind/tftest/od_pet/data/model.ckpt.meta
stderr:


ok

### Configuring the Object Detection Pipline

In [34]:
- shell: |
    . ~/.zshrc
    cp {{od_project_prefix}}/object_detection/samples/configs/faster_rcnn_resnet101_pets.config \
        {{od_pet_project_path}}/data/faster_rcnn_resnet101_pets.config
    
    sed -i "s|PATH_TO_BE_CONFIGURED|{{od_pet_project_path}}/data|g" \
        {{od_pet_project_path}}/data/faster_rcnn_resnet101_pets.config
  args:
    executable: /bin/zsh

Use hosts: tf	Use gather_facts: no	
TASK [192.168.10.238: TASK: command] 
stdout:

stderr:


ok

## Running the Training Job
每10分钟输出一个checkpoint,运行训练后1个小时停止训练

In [2]:
- shell: |
    . ~/.zshrc
    mkdir -p {{od_pet_project_path}}/models/train
    python object_detection/train.py \
        --logtostderr \
        --pipeline_config_path={{ od_pet_project_path }}/data/faster_rcnn_resnet101_pets.config \
        --train_dir={{od_pet_project_path}}/models/train
        
  args:
    chdir: '{{ od_project_prefix }}'
    executable: /bin/zsh
  async: 3800
  poll: 60

Use hosts: tf	Use gather_facts: no	


TASK [192.168.10.238: TASK: command] 
{
  "failed": true,
  "msg": "async task did not complete within the requested time",
  "_ansible_no_log": false,
  "changed": false
}

error: 2

## Running the Evaluation Job

In [3]:
- shell: |
    . ~/.zshrc
    mkdir -p {{od_pet_project_path}}/models/eval
    python object_detection/eval.py \
        --logtostderr \
        --pipeline_config_path={{ od_pet_project_path }}/data/faster_rcnn_resnet101_pets.config \
        --checkpoint_dir={{od_pet_project_path}}/models/train \
        --eval_dir={{od_pet_project_path}}/models/eval
        
  args:
    chdir: '{{ od_project_prefix }}'
    executable: /bin/zsh
  async: 300
  poll: 60

Use hosts: tf	Use gather_facts: no	


TASK [192.168.10.238: TASK: command] 
{
  "failed": true,
  "msg": "async task did not complete within the requested time",
  "_ansible_no_log": false,
  "changed": false
}

error: 2